# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import imutils
import matplotlib.pyplot as plt


from collections import defaultdict
from io import StringIO

from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.11.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib auto

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
#http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
#MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09'
MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09'
#MODEL_NAME = 'ssd_inception_v2_coco_2018_01_28'
#http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gzMODE
#MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [ ]:
print("Is file downloaded? %s " % os.path.isfile(MODEL_FILE))
if not os.path.isfile(MODEL_FILE):
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [ ]:
def removeBackground(frame):
	global fgbg
	frameMasked = fgbg.apply(frame.copy())

	im2, contours, hierarchy = cv2.findContours(frameMasked, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

	#framedMaskedWithoutContours = removeBadContours(frameMasked)
	framedMaskedWithoutContours = frameMasked
	im2, contours, hierarchy = cv2.findContours(framedMaskedWithoutContours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cv2.drawContours(framedMaskedWithoutContours, contours, -1, (255, 255, 255), 3)
	framedMaskedWithoutContours = cv2.cvtColor(framedMaskedWithoutContours, cv2.COLOR_GRAY2RGB)


	return cv2.bitwise_and(frame, framedMaskedWithoutContours)
	#return frameMaskedAux

## Load a (frozen) Tensorflow model into memory.

In [ ]:

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)


# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
avg = None
def motionDetection(frame):
 global avg
 # resize the frame, convert it to grayscale, and blur it
 
 gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 gray = cv2.GaussianBlur(gray, (21, 21), 0)
 # if the average frame is None, initialize it
 if avg is None:
  print("[INFO] starting background model...")
  avg = gray.copy().astype("float")
  return False
                            
 # accumulate the weighted average between the current frame and
 # previous frames, then compute the difference between the current
 # frame and running average
 cv2.accumulateWeighted(gray, avg, 0.5)
 frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
 # threshold the delta image, dilate the thresholded image to fill
 # in holes, then find contours on thresholded image
 thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
 thresh = cv2.dilate(thresh, None, iterations=2)
 im2, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
 return contours != []                        

In [ ]:
def run_inference_for_single_image(frame_expanded, graph):
        # Perform the actual detection by running the model with the image as input
      (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

      output_dict = {
           "detection_classes":np.squeeze( classes).astype(np.int32),
           "detection_boxes": boxes[0],
           "detection_scores": np.squeeze(scores)
       }     
      return output_dict

In [ ]:
# blocks until it finds the next video to analyse
import os
import time

def isValidVideoFile(file):
     cap = cv2.VideoCapture(file)
     totalFrameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
     cap.release()
     return totalFrameCount >0

def wait_for_video( directory, time_limit=3600, check_interval=60 ):
    '''Return next video file to process, if not keep checking once every check_interval seconds for time_limit seconds.
    time_limit defaults to 1 hour
    check_interval defaults to 1 minute
    '''

    now = time.time()
    last_time = now + time_limit

    while time.time() <= last_time:
        print("Searching for new camera uploads")
        for root, dirs, files in os.walk(directory):
           files = [ fi for fi in files if fi.endswith(".mp4") and not fi.startswith(".")  ]
           for file in files:
                filePath = os.path.join(root, file)
               
                if not os.path.isfile(filePath + '.processed') and \ 
                        os.path.getsize(filePath) > 0 and \
                        isValidVideoFile(filePath):
                    return filePath
 
        # Wait for check interval seconds, then check again.
        time.sleep( check_interval )

    return None

#wait_for_video( "/data/videos/incoming", 3600, 1 )

In [ ]:
def moveVideoToArchive(file):
    open(file + '.processed', 'w')

In [ ]:
def detectPerson(output_dict, category_index):
    result = False
    classes = output_dict['detection_classes']  
    for i in range(len(classes)):
        score = output_dict['detection_scores'][i]
        
        label = category_index.get(classes[i])
        if (score>0.5 and label['name'] == 'person' ):
            result = True
            break
            
    return result


In [ ]:
import configparser

Config = configparser.ConfigParser()
Config.read("./config.ini")

def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    print(options)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1


from slackclient import SlackClient
import io
slack_token = ConfigSectionMap("Slack")['secrettoken']
channel_id = ConfigSectionMap("Slack")['channelid']
sc = SlackClient(slack_token)


def notifySlack(plt, image_np, image_no_background):
    plt.imsave("/data/image_with_background.png", image_np)
    plt.imsave("/data/image_no_background.png", image_no_background)
    sc.api_call(
      "chat.postMessage",
      channel=channel_id,
      text="Unidentified person detected in kitchen for file={}".format(file)

    )


    with open('/data/image_with_background.png', 'rb') as f:
        sc.api_call(
            "files.upload",
            channels=channel_id,
            filename='snapshot1.png',
            title='Detected Person',
            initial_comment='Detected person by webcam. Is it anyone you know?',
            file=io.BytesIO(f.read())
        )
    with open('/data/image_no_background.png', 'rb') as f:
        sc.api_call(
            "files.upload",
            channels=channel_id,
            filename='snapshot2.png',
            title='Detected Person',
            initial_comment='Detected person by webcam. Is it anyone you know?',
            file=io.BytesIO(f.read())
        )

        
    with open(file, 'rb') as f:
        sc.api_call(
            "files.upload",
            channels=channel_id,
            filename='$file',
            title='Video with detected person',
            initial_comment='Video with detected person',
            file=io.BytesIO(f.read())
        )


In [ ]:
FRAMES_TO_SKIP = 19
fgbg = None
def processVideoFile(file):
     global avg
     global fgbg
     cap = cv2.VideoCapture(file)
     totalFrameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
     if totalFrameCount == 0:
            print("Video file=%s not yet ready for processing. Skipping." % file)
            return False
     print("Processing video file with %d frames" % totalFrameCount)

     # reset motion detection for new video file
     avg = None                    
     motionDetected = False
     fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
     while True:
        ret, image_np = cap.read()
        currentFrame = cap.get(cv2.CAP_PROP_POS_FRAMES)
       
        print ("Processing frame %d of %d for video file=%s" % (currentFrame, totalFrameCount, file)) 
        if not ret:
            cap.release()
            break
        image_np_reduced = imutils.resize(image_np, width=500)   
        frameWithoutBackground = removeBackground(image_np_reduced)
        # only apply the motion detection algorithm if motion hasn't yet been detected
        if (motionDetection(image_np_reduced)):
            print("motion detected - analysing image with Tf")
             # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
          
            image_np_expanded = np.expand_dims(frameWithoutBackground, axis=0)
             # Actual detection.
            output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
            # Visualization of the results of a detection.
            if detectPerson(output_dict,category_index):
                print ("detected person")
                image_np.setflags(write=1)
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np_reduced,
                    output_dict['detection_boxes'],
                    output_dict['detection_classes'],
                    output_dict['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=0.40)


                #plt.figure(figsize=IMAGE_SIZE)
                #plt.imshow(image_np)
                notifySlack(plt, image_np_reduced,frameWithoutBackground)

                break
            
        # skips frames as the rate of the camera is 20 fps
        nextFrame = min(currentFrame + FRAMES_TO_SKIP, totalFrameCount)
        cap.set(cv2.CAP_PROP_POS_FRAMES, nextFrame)  
     return True

In [ ]:
while True:

  file = wait_for_video( "/data/videos/incoming", 3600 * 354, 1 ) 

  if file is None:
    break
  
  result = processVideoFile(file)
  
  if result:
        moveVideoToArchive(file)